<a href="https://colab.research.google.com/github/AndrewFatula/Simple-CNNs/blob/master/mnist_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from keras import models 
from keras import layers
from keras.utils import np_utils
from keras import backend as k_backend
from keras.datasets import mnist
from keras import regularizers
import time



tf.enable_eager_execution()


In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 1, 28, 28)
x_test = x_test.reshape(x_test.shape[0], 1, 28, 28)

x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

x_train = (x_train - np.mean(x_train))/ x_train.std()
x_test = (x_test - np.mean(x_test))/ x_test.std()

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

batch_size = 512
num_classes = 10
n_filters = 64
n_fc1 = 512
n_fc2 = 128
pool = (4,4)
kernel_size1 = (5,5)
kernel_size2 = (4,4)
kernel_size3 = (2,2)
input_shape1 = (1, 28, 28)
input_shape2 = (n_filters, 23, 23)
input_shape3 = (n_filters, 11, 11)
n_epochs = 40



In [0]:
mnist_net = tf.keras.Sequential([
tf.keras.layers.Conv2D(n_filters, kernel_size1, input_shape=input_shape1, data_format="channels_first", activation="relu", kernel_regularizer=regularizers.l2(0.002)),
tf.keras.layers.Dropout(0.3),
tf.keras.layers.MaxPooling2D(pool_size = pool, strides = (2,2), data_format="channels_first"),
tf.keras.layers.Conv2D(n_filters*2, kernel_size3, input_shape=input_shape3, data_format="channels_first", activation="relu", kernel_regularizer=regularizers.l2(0.002)),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.MaxPooling2D(pool_size = (2,2), data_format="channels_first"),
tf.keras.layers.Flatten(data_format="channels_first"),
tf.keras.layers.Dense(n_fc1, activation="sigmoid", kernel_regularizer=regularizers.l2(0.002)),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(n_fc2, activation="sigmoid", kernel_regularizer=regularizers.l2(0.002)),
tf.keras.layers.Dropout(0.1),
tf.keras.layers.Dense(num_classes, kernel_regularizer=regularizers.l2(0.002))
])



In [0]:

(mnist_images, mnist_labels), _ = tf.keras.datasets.mnist.load_data()


In [0]:
def train_test_split(x, y):
  data = list(zip(x,y))
  np.random.shuffle(data)
  train_data = data[0:50000]
  test_data = data[50000:]
  return zip(*train_data), zip(*test_data)

def get_batches(x, y, batch_size):
  length = np.shape(x)[0]
  n_of_entries = int(length/batch_size)
  batched_x = []
  batched_y = []

  for i in range(n_of_entries):
    batch_x = []
    batch_y = []
    for j in range(batch_size):
      batch_x.append(x[i*batch_size + j])
      batch_y.append(y[i*batch_size + j])

    batched_x.append(batch_x)
    batched_y.append(batch_y)

  return batched_x, batched_y     

def get_accuracy(labels, images, model):
  length = len(labels)
  step_length = 1000
  total_correct = 0
  images = np.expand_dims(images, 1)
  for i in range(step_length):
    step_labels = labels[i*step_length : (i+1)*step_length]
    step_logits = model(images[step_length*i : step_length*(1+i)])
    pred_numbers = np.argmax(step_logits, axis = 1)
    total_correct += np.sum(np.where(step_labels == pred_numbers,1,0))
  return total_correct/length

(train_images, train_labels), (test_images, test_labels) = train_test_split(mnist_images, mnist_labels)

train_images = np.float32(np.array(train_images)/255.0)
test_images = np.float32(np.array(test_images)/255.0)

test_labels = np.array(test_labels)
test_images = np.array(test_images)



train_batched_images, train_batched_labels = get_batches(train_images, train_labels, batch_size)


In [0]:

start = time.localtime(time.time())

optimizer = tf.train.AdamOptimizer(0.0005)

for i in range(n_epochs):
  train_batches = list(zip(train_batched_images, train_batched_labels))
  np.random.shuffle(train_batches)
  for batch_x, batch_y in train_batches:
    batch_x = np.expand_dims(batch_x, axis = 1)
    with tf.GradientTape() as tape:
      logits = mnist_net(batch_x, training=True)
    
      loss_value = tf.losses.sparse_softmax_cross_entropy(batch_y, logits)
      grads = tape.gradient(loss_value, mnist_net.trainable_variables)
    optimizer.apply_gradients(zip(grads, mnist_net.trainable_variables),
                              global_step=tf.train.get_or_create_global_step())
  
  
  print(get_accuracy(test_labels, test_images, mnist_net))

end = time.localtime(time.time())
start_in_sec = start[3]*3600 + start[4]*60 + start[5]
end_in_sec = end[3]*3600 + end[4]*60 + end[5]


all_time_min = int((end_in_sec-start_in_sec)/60)
all_time_sec = (end_in_sec-start_in_sec)%60
if all_time_min < 10:
	if all_time_sec < 10:
		print('0%s:0%s' % (all_time_min, all_time_sec ))
	else:
		print('0%s:%s' % (all_time_min, all_time_sec ))	
else:
	if all_time_sec < 10:
		print('%s:0%s' % (all_time_min, all_time_sec ))
	else:
		print('%s:%s' % (all_time_min, all_time_sec ))	
		
    



0.944
0.968
